<div>
<img src="https://camo.githubusercontent.com/473dd9f992924d27457650251786464f72e54121ac6e9210add0f483ca849277/68747470733a2f2f692e696d6775722e636f6d2f3765523750616e2e706e67" width="40%">  
</div>

# Distributed Bloom for Text Classification using Prompt Tuning

In this example, we show how to use [prompt tuning](https://aclanthology.org/2021.emnlp-main.243.pdf) to adapt a test 6B version of the [BLOOM](https://huggingface.co/bigscience/bloom) model for a specific downstream task. We will run this model in a decentralized fashion using [Petals](https://github.com/bigscience-workshop/petals). Petals servers will maintain the BLOOM blocks (they are kept unchanged during adaptation), and the gradient descent will learn a few prefix tokens stored on a Petals client.

We will adapt the BLOOM model for the classification task using the [SST-2 dataset](https://nlp.stanford.edu/sentiment/). This dataset is a binary classification task, where the goal is to predict whether a sentence is positive or negative. The SST-2 dataset is a subset of the Stanford Sentiment Treebank, and it is available in the [Hugging Face Datasets](https://huggingface.co/datasets) library.

To open this notebook in colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bigscience-workshop/petals/blob/main/examples/prompt-tuning-sst2.ipynb)

First, we have to prepare all dependencies.

In [ ]:
import subprocess
import sys


IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    subprocess.run("git clone https://github.com/bigscience-workshop/petals", shell=True)
    subprocess.run("pip install -r petals/requirements.txt", shell=True)
    subprocess.run("pip install datasets wandb", shell=True)

    try:
        subprocess.check_output(["nvidia-smi", "-L"])
    except subprocess.CalledProcessError as e:
        subprocess.run("rm -r /usr/local/cuda/lib64", shell=True)

    sys.path.insert(0, './petals/')
else:
    sys.path.insert(0, "..")

In [ ]:
import os
import sys
 
import torch
import transformers
import wandb
from datasets import load_dataset, load_metric
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_scheduler

# Import a Petals model
from src.client.remote_model import DistributedBloomForSequenceClassification

Let's set some hyperparameters for training:

In [ ]:
MODEL_NAME = ... # select model you like
INITIAL_PEERS = [...] # add your peers adresses here, like "/ip4/192.168.1.2/tcp/31000/p2p/Qma...."
NUM_PREFIX_TOKENS = 16
DEVICE = 'cpu'
BATCH_SIZE = 4
LR = 1e-2
WEIGHT_DECAY = 0.0
NUM_SAMPLES = 1000
NUM_EPOCHS = 3
SEED = 42
MODEL_MAX_LENGTH = 64
TUNING_MODE = 'ptune' # choose between ['ptune', 'deep_ptune'] 

Prepare tokenizer and distributed model, connect it to servers.

In [ ]:
tokenizer = transformers.BloomTokenizerFast.from_pretrained(MODEL_NAME)
tokenizer.padding_side = 'right'
tokenizer.model_max_length = MODEL_MAX_LENGTH
model = DistributedBloomForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    initial_peers=INITIAL_PEERS, 
    pre_seq_len=NUM_PREFIX_TOKENS, 
    tuning_mode=TUNING_MODE
).to(DEVICE)

Let's prepare the SST-2 dataset. We need just one preprocessing function to tokenize the dataset.

In [ ]:
task = 'sst2'

dataset = load_dataset("glue", task)

def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx", "attention_mask"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"].shuffle(seed=SEED)
valid_dataset = tokenized_datasets["validation"].shuffle(seed=SEED)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

To check training, we need a metric function. For SST-2 task is accuracy. We will load it from the datasets library.

In [ ]:
metric = load_metric('glue', task)

def eval_metrics(model, dataloader, device='cpu'):
    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    model.train()
    return metric.compute()

Before setting up optimizers, check the model parameters that will be trained.

In [ ]:
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n, p.requires_grad, p.device)

The optimizer will only work on **prompts**, they are only trainable parameters. Let's initialize optimizer and learning rate scheduler.

In [ ]:
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)
)

Let's initialize wandb for logging and start the training loop!

In [ ]:
wandb.init(
    project="bloom-sst-2",
    config={
        "num_epochs": NUM_EPOCHS,
        "num_samples": NUM_SAMPLES,
        "batch_size": BATCH_SIZE,
        "learning_rate": LR,
        "weight_decay": WEIGHT_DECAY,
        "num_prefix_tokens": NUM_PREFIX_TOKENS,
        "model_name": MODEL_NAME,
        "seed": SEED,
    }
)

for epoch in range(NUM_EPOCHS):
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        model.train()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({"Train Loss": loss})

    accuracy = eval_metrics(model, valid_dataloader, device=DEVICE)
    wandb.log({"Valid Accuracy": accuracy}, commit=False)

Our model have been trained!